In [15]:
%reload_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup
import requests
import time
import random
import pickle as pkl
from funcs_proxy import *
from funcs_scrape_pdf import *
import pprint
pp = pprint.PrettyPrinter()

## Proxy info

In [2]:
url_proxy = 'https://free-proxy-list.net/'
timeout = 20
proxies_n = 15
proxies = get_proxies(url_proxy, proxies_n)

In [4]:
# proxies
# response = requests.get(url_proxy, proxies={"http": proxies[2], "https": proxies[2]}) #, timeout=timeout

## List of philosophical domains

In [3]:
url_domains = 'http://philosophyideas.com/search/themes.asp'
domains = get_philosophical_domains(url_domains)
pp.pprint(domains)

['1. Philosophy',
 '2. Reason',
 '3. Truth',
 '4. Formal Logic',
 '5. Theory of Logic',
 '6. Mathematics',
 '7. Existence',
 '8. Modes of Existence',
 '9. Objects',
 '10. Modality',
 '11. Knowledge Aims',
 '12. Knowledge Sources',
 '13. Knowledge Criteria',
 '14. Science',
 '15. Nature of Minds',
 '16. Persons',
 '17. Mind and Body',
 '18. Thought',
 '19. Language',
 '20. Action',
 '21. Aesthetics',
 '22. Metaethics',
 '23. Ethics',
 '24. Applied Ethics',
 '25. Society',
 '26. Natural Theory',
 '27. Natural Reality',
 '28. God',
 '29. Religion']


## Get philosophical domain ideas
***Domain strings match an element in domains list printed above***

In [13]:
domain = '14. Science'
idea_ids_desired = [id.strip() for id in open(f"./idea_ids_{domain.replace(' ', '_')}.txt", 'r')]
ideas = []
invalid_idea_ids = []

In [14]:
for idx, id in enumerate(idea_ids_desired):
    
    url_phil = f'http://philosophyideas.com/search/idea_detail.asp?find=theme&themenumber=2015&visit=1&return=yes&ID={id}&gistsfor=theme&theme_alpha=yes&ThemeNumber=&area=&source=single&area_no='
    
    r = requests.get(url_phil)
#     if idx < 5:
#         r = requests.get(url_phil)
#     elif idx % 50 == 0:
#         proxy = get_proxy(url_proxy, proxies, timeout)
#         r = requests.get(url_phil, proxies=proxy)
#         print(proxy)
#     else:
#         r = requests.get(url_phil, proxies=proxy)
    
    soup = BeautifulSoup(r.content, 'lxml') 
    table = soup.find('div', attrs = {'class': 'txt'})
    
    headers = table.find_all_next('p', attrs = {'class': 'para2'})
    
    if len(headers) == 0:
        invalid_idea_ids.append(id)
        print(id)
        continue
        
    headers = [header.get_text().strip() for header in headers]
    gist_idx = headers.index('Gist of Idea')
    full_idea_idx = headers.index('Full Idea')
    source_idx = headers.index('Source')
    
    datums = table.find_all_next('p', attrs = {'class': 'text'})
    
    gotcha = {}
    gotcha['gist'] = datums[gist_idx].get_text().strip()
    gotcha['full_idea'] = datums[full_idea_idx].get_text().strip()
    gotcha['source'] = datums[source_idx].get_text().strip()
    
    ideas.append(gotcha)
    
    if idx != len(idea_ids_desired) - 1:
        sleepy_time = random.uniform(0, 0.5)
        if idx % 10 == 0:
            print(idx)
#             print(sleepy_time)
        time.sleep(sleepy_time)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650


In [16]:
len(ideas), len(invalid_idea_ids)

(660, 0)

In [17]:
pkl.dump(ideas, open(f"./ideas_{domain.replace(' ', '_')}.pickle", 'wb'))
# pkl.dump(ideas, open("ideas_truth_idx_0_to_299.pickle", "wb" ))

In [18]:
ideas_load = pkl.load(open(f"./ideas_{domain.replace(' ', '_')}.pickle", 'rb'))

In [21]:
ideas_load[10]

{'gist': 'We can know by demonstration, which is a scientific deduction leading to understanding',
 'full_idea': "We know things through demonstration, by which I mean a scientific deduction, and by 'scientific' I mean a deduction by possessing which we understand something.",
 'source': 'Aristotle (Posterior Analytics [c.327 BCE],  71b17)'}

In [42]:
ideas_load1 = pkl.load(open("ideas_truth_idx_0_to_299.pickle", "rb"))
ideas_load2 = pkl.load(open("ideas_truth_idx_300_to_end.pickle", "rb"))

In [43]:
ideas_load1.extend(ideas_load2)

In [44]:
len(ideas_load1)

588

In [45]:
domain2 = '3. Truth'
pkl.dump(ideas_load1, open(f"./ideas_{domain2.replace(' ', '_')}.pickle", 'wb'))

In [27]:
ideas_load = pkl.load(open(f"./ideas_{domain2.replace(' ', '_')}.pickle", 'rb'))

In [31]:
ideas_load[-1]

{'gist': "Minimal theories of truth avoid ontological commitment to such things as 'facts' or 'reality'",
 'full_idea': "Minimalist theories of truth are those which involve minimum ontological commitment, avoiding references to 'reality' or 'facts' or 'what works', preferring to refer to formal relationships within language.",
 'source': 'PG (Db (ideas) [2031])'}

In [34]:
ideas_load[-1]['gist']

"Minimal theories of truth avoid ontological commitment to such things as 'facts' or 'reality'"

In [37]:
type(ideas_load)

list

## Print philosophical ideas to submit to model
***Domain strings match an element in domains list printed above***

In [46]:
domains = ('3._Truth', '14._Science')
key_for_print = 'gist'

In [48]:
with open('./ideas_submit2Model.txt', 'w') as f:
    for idx, domain in enumerate(domains):
#         with open(f"./ideas_{domain.replace(' ', '_')}.pickle", 'rb') as ideas:
        ideas = pkl.load(open(f"./ideas_{domain.replace(' ', '_')}.pickle", 'rb'))
        for idea in ideas[:-1]:
            f.write('%s\n' % idea[key_for_print])
        if idx == len(domains) - 1:
            f.write('%s' % ideas[-1][key_for_print])
        else:
            f.write('%s\n' % ideas[-1][key_for_print])

## Test some of the generated stuff

In [56]:
does_this_exist = 'combined to specify the same thing'
does_this_exist = 'predicate is expressive of the truth predicate'

In [57]:
ideas_key = [idea[key_for_print] for idea in ideas]

In [60]:
for idea in ideas_key:
    if does_this_exist in idea:
        print(idea)

In [61]:
does_this_exist

'predicate is expressive of the truth predicate'

Do a more comprehensive study of the similarity of generated to actual ideas